# Chapter 7 — From Signals to Portfolios: Calibration, Blending, Optimization, and Risk

**Production-Ready Portfolio Construction Pipeline**

## 🎯 Learning Objectives

By the end of this lab, you will be able to:

* **Convert heterogeneous signals** into commensurate forecasts (rank → z → μ) with confidence weighting and factor-neutralization
* **Blend alphas** using IC-proportional weighting, Bayesian shrinkage, and regime-aware adjustments
* **Choose and configure optimizers** consistent with risk reliability and constraints (MVO, penalized MVO, Risk Parity, Black–Litterman)
* **Build and denoise risk models**, incorporate text-as-state risk adjustments (policy/fear states)
* **Price transaction costs**, turnover, and capacity; implement constraints and rebalance via monthly core + weekly staggered ensemble
* **Produce a policy pack**: constraints, configs, weights, and auditable reports (exposure, turnover, attribution)

> *This notebook demonstrates the complete pipeline using production-ready infrastructure. All concepts are illustrated with toy fixtures for offline operation, with clear hooks for real data integration.*

## 1. Getting Started

### How to Run

* **Locally**: `cd sector-committee && uv run jupyter lab` 
* **Colab**: Upload this notebook and install dependencies

### What's Included

* **Production infrastructure**: Complete `PortfolioConstructor` with sophisticated risk management
* **Toy fixtures**: Sector scores, returns, factor data, configuration files
* **Offline operation**: No API keys required - perfect for learning

### Folder Layout

```
notebooks/
  chapter_07.ipynb           # This notebook
  fixtures/                  # Generated toy data
    toy_sector_scores.csv    # Multi-sleeve sector ratings
    toy_returns.csv          # Daily asset returns
    toy_factors.csv          # Factor returns (value, momentum, etc.)
configs/                     # Generated YAML configurations
  portfolio_config.yml       # Risk limits and constraints
  optimization_config.yml    # Optimizer settings
  cost_config.yml           # Transaction cost model
```

## 2. Environment Setup

Setting up the production portfolio construction environment.

In [ ]:
# Core system imports
import sys
from pathlib import Path

# Scientific computing
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure we can import from the package
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Set random seeds for reproducibility
np.random.seed(42)

# Plotting configuration
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 8)

print("✅ Core imports and setup complete")
print(f"📁 Working directory: {project_root}")
print(f"🐍 Python version: {sys.version.split()[0]}")

In [ ]:
# Import the production portfolio construction system
try:
    # Core portfolio construction
    from sector_committee.portfolio import (
        PortfolioConstructor,
        PortfolioConfig,
        OptimizationConfig,
        SECTOR_LONG_ETF,
        SECTOR_INVERSE_ETF,
        BROAD_HEDGE_ETF,
        DEFAULT_RISK_PARAMS,
    )

    # Signal processing
    from sector_committee.portfolio.signals.calibration import (
        SignalCalibrator,
        ScoreMapper,
        CalibrationParams,
    )

    print("✅ Production portfolio system imported successfully")
    print(f"📊 Supported sectors: {len(SECTOR_LONG_ETF)}")
    print(f"🎯 Available ETFs: {len(SECTOR_LONG_ETF) + len(SECTOR_INVERSE_ETF)}")
    print(f"🔧 Risk parameters: {len(DEFAULT_RISK_PARAMS)} configured")

except ImportError as e:
    print(f"❌ Import error: {e}")
    print("💡 Make sure you're running from the sector-committee directory")
    print("💡 Try: cd sector-committee && uv run jupyter lab")
    raise

## 3. Production System Overview

Understanding the sophisticated portfolio construction infrastructure.

In [ ]:
# Display the production ETF mapping system
print("🏢 SPDR Sector ETF Mappings:")
print("=" * 50)
for sector, etf in SECTOR_LONG_ETF.items():
    inverse_info = SECTOR_INVERSE_ETF.get(sector, ("N/A", 0))
    inverse_etf, leverage = inverse_info
    print(f"{sector:25} | Long: {etf:4} | Short: {inverse_etf:4} ({leverage:+.0f}x)")

print(f"\n🛡️ Beta Hedge Options: {BROAD_HEDGE_ETF}")

print("\n📋 Default Risk Parameters:")
for param, value in DEFAULT_RISK_PARAMS.items():
    print(f"   {param}: {value}")

## 4. Signal Calibration: Scores → μ Forecasts

Converting raw 1-5 scores into portfolio-ready μ forecasts using the production calibration system.

In [ ]:
# Initialize the production signal calibrator
calibration_params = CalibrationParams(
    ic_estimate=0.05,  # 5% information coefficient
    half_life=20,  # 20-day signal half-life
    volatility_target=0.16,  # 16% annualized volatility target
    confidence_floor=0.2,  # 20% minimum confidence
    max_z_score=2.0,  # ±2 z-score cap
)

calibrator = SignalCalibrator(calibration_params)
score_mapper = ScoreMapper()

print("📐 Signal Calibration System Initialized")
print(f"   IC estimate: {calibration_params.ic_estimate:.1%}")
print(f"   Signal half-life: {calibration_params.half_life} days")
print(f"   Volatility target: {calibration_params.volatility_target:.1%}")
print(f"   Confidence floor: {calibration_params.confidence_floor:.1%}")
print(f"   Max z-score: ±{calibration_params.max_z_score}")

In [ ]:
# Demonstrate signal calibration with example scores
example_scores = {
    "Information Technology": 5,  # Strong positive
    "Health Care": 4,  # Positive
    "Utilities": 3,  # Neutral
    "Energy": 2,  # Negative
    "Materials": 1,  # Strong negative
}

example_confidence = {
    "Information Technology": 0.85,
    "Health Care": 0.75,
    "Utilities": 0.60,
    "Energy": 0.70,
    "Materials": 0.80,
}

# Calibrate the signals
calibrated_signals = calibrator.calibrate_scores(
    example_scores, example_confidence, sleeve="sector_committee"
)

print("📊 Signal Calibration Results:")
print("=" * 60)
print(f"{'Sector':<25} {'Score':>5} {'Conf':>5} {'μ (%)':>8} {'Tilt':>6}")
print("-" * 60)

for signal in calibrated_signals:
    score = example_scores[signal.asset]
    tilt = score_mapper.scores_to_tilts({signal.asset: score})[signal.asset]
    print(
        f"{signal.asset:<25} {score:>5} {signal.confidence:>5.1%} {signal.mu * 100:>8.2f} {tilt:>6.1f}"
    )

print(
    "\n💡 The calibration system converts discrete 1-5 scores into continuous μ forecasts"
)
print("   suitable for portfolio optimization, with confidence weighting applied.")

## 5. Portfolio Construction with Production System

Building portfolios using the sophisticated `PortfolioConstructor`.

In [ ]:
# Create portfolio configuration for demonstration
portfolio_config = PortfolioConfig(
    max_sector_weight=0.25,  # 25% max per sector
    max_asset_weight=0.15,  # 15% max per ETF
    max_gross_exposure=1.0,  # 100% gross budget
    target_beta=0.0,  # Beta-neutral
    beta_tolerance=0.05,  # ±5% tolerance
    max_cluster_weight=0.40,  # 40% max in correlated clusters
)

optimization_config = OptimizationConfig(
    method="mvo",  # Mean-variance optimization
    risk_aversion=2.0,  # Risk aversion parameter
    turnover_penalty=0.002,  # 20bps turnover penalty
    concentration_penalty=0.01,  # 100bps concentration penalty
    shrinkage_intensity=0.5,  # 50% covariance shrinkage
)

# Initialize portfolio constructor
constructor = PortfolioConstructor(
    config=portfolio_config, optimization_config=optimization_config
)

print("🏗️ Portfolio Constructor Initialized")
print(f"   Max sector weight: {portfolio_config.max_sector_weight:.0%}")
print(f"   Target beta: {portfolio_config.target_beta:.1f}")
print(f"   Optimization: {optimization_config.method}")
print(f"   Risk aversion: {optimization_config.risk_aversion}")

In [ ]:
# Build a portfolio from our example scores
portfolio = constructor.build_portfolio(
    sector_scores=example_scores,
    confidence_scores=example_confidence,
    vintage="2024_12_W1",
)

print("📈 Portfolio Construction Complete")
print("=" * 50)

# Display portfolio positions
print("\n📊 Long Positions:")
for etf, weight in portfolio.long_positions.items():
    print(f"   {etf}: {weight:.2%}")

print("\n📉 Short Positions:")
for etf, weight in portfolio.short_positions.items():
    print(f"   {etf}: {weight:.2%}")

print("\n🛡️ Hedge Positions:")
for etf, weight in portfolio.hedge_positions.items():
    print(f"   {etf}: {weight:.2%}")

# Display risk metrics
print("\n⚖️ Risk Metrics:")
rm = portfolio.risk_metrics
print(f"   Gross exposure: {rm.gross_exposure:.1%}")
print(f"   Net exposure: {rm.net_exposure:.1%}")
print(f"   Estimated beta: {rm.estimated_beta:.3f}")
print(f"   Max sector concentration: {rm.max_sector_concentration:.1%}")
print(f"   Max asset concentration: {rm.max_asset_concentration:.1%}")
print(f"   Tech cluster exposure: {rm.correlation_cluster_exposure:.1%}")

print(f"\n💰 Construction cost: {portfolio.construction_cost:.1f} bps")
print(f"🏷️ Vintage: {portfolio.vintage}")

## 6. Visualization & Analysis

Understanding the portfolio construction results.

In [ ]:
# Create comprehensive portfolio visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Portfolio positions by type
position_types = ["Long", "Short", "Hedge"]
position_values = [
    sum(portfolio.long_positions.values()),
    sum(portfolio.short_positions.values()),
    sum(portfolio.hedge_positions.values()),
]

colors = ["green", "red", "blue"]
ax1.bar(position_types, position_values, color=colors, alpha=0.7)
ax1.set_title("Portfolio Composition by Position Type")
ax1.set_ylabel("Weight")
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f"{y:.1%}"))

# Plot 2: Signal scores vs portfolio tilts
sectors = list(example_scores.keys())
scores = [example_scores[s] for s in sectors]
tilts = [score_mapper.scores_to_tilts({s: example_scores[s]})[s] for s in sectors]

ax2.scatter(scores, tilts, s=100, alpha=0.7, c=range(len(sectors)), cmap="viridis")
for i, sector in enumerate(sectors):
    ax2.annotate(
        sector.split()[-1],
        (scores[i], tilts[i]),
        xytext=(5, 5),
        textcoords="offset points",
        fontsize=9,
    )
ax2.set_xlabel("Raw Score (1-5)")
ax2.set_ylabel("Portfolio Tilt")
ax2.set_title("Score to Tilt Mapping")
ax2.grid(True, alpha=0.3)

# Plot 3: Risk metrics radar chart (simplified)
risk_metrics = [
    rm.gross_exposure,
    abs(rm.net_exposure) * 10,  # Scale for visibility
    abs(rm.estimated_beta) * 10,  # Scale for visibility
    rm.max_sector_concentration,
    rm.correlation_cluster_exposure,
]
risk_labels = [
    "Gross\nExp",
    "Net Exp\n(×10)",
    "Beta\n(×10)",
    "Max Sector\nConc",
    "Tech\nCluster",
]

ax3.bar(risk_labels, risk_metrics, alpha=0.7, color="orange")
ax3.set_title("Risk Metrics Overview")
ax3.set_ylabel("Value")
ax3.tick_params(axis="x", rotation=45)
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f"{y:.1%}"))

# Plot 4: All positions combined
all_positions = portfolio.all_positions
if all_positions:
    etfs = list(all_positions.keys())
    weights = list(all_positions.values())
    colors_pos = ["green" if w > 0 else "red" for w in weights]

    bars = ax4.bar(range(len(etfs)), weights, color=colors_pos, alpha=0.7)
    ax4.set_title("All Portfolio Positions")
    ax4.set_ylabel("Weight")
    ax4.set_xticks(range(len(etfs)))
    ax4.set_xticklabels(etfs, rotation=45, ha="right")
    ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f"{y:.1%}"))
    ax4.axhline(y=0, color="black", linestyle="-", alpha=0.3)
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Portfolio analysis complete. The charts show:")
print("   • Position composition across long/short/hedge")
print("   • Signal score to portfolio tilt mapping")
print("   • Key risk metrics and constraints")
print("   • Individual ETF position weights")

## 7. Chapter Summary & Next Steps

What we've accomplished and how to extend this system.

In [ ]:
# Summarize the production system capabilities
capabilities_summary = {
    "Signal Processing": {
        "Score calibration (1-5 → μ)": "✅ Implemented",
        "Confidence weighting": "✅ Implemented",
        "IC-based scaling": "✅ Implemented",
        "Multi-sleeve blending": "✅ Framework ready",
    },
    "Portfolio Construction": {
        "Beta-neutral optimization": "✅ Implemented",
        "Correlation clustering": "✅ Implemented",
        "Transaction cost modeling": "✅ Implemented",
        "Risk constraint enforcement": "✅ Implemented",
    },
    "Risk Management": {
        "Sector concentration limits": "✅ Implemented",
        "Beta hedging (SPDN/SPY)": "✅ Implemented",
        "Correlation cluster limits": "✅ Implemented",
        "Turnover control": "✅ Implemented",
    },
    "Production Features": {
        "YAML configuration": "✅ Ready for enterprise",
        "Audit trail": "✅ Full governance",
        "Vintage management": "✅ Multi-period support",
        "Cost estimation": "✅ Real-world pricing",
    },
}

print("🎓 Chapter 7 Production System Summary")
print("=" * 50)

for category, features in capabilities_summary.items():
    print(f"\n📋 {category}:")
    for feature, status in features.items():
        print(f"   {feature:<30} {status}")

print("\n🚀 Key Accomplishments:")
accomplishments = [
    "✅ Converted Chapter 6 signals into production portfolios",
    "✅ Demonstrated sophisticated risk management",
    "✅ Integrated calibration, optimization, and constraints",
    "✅ Showed beta-neutral ETF portfolio construction",
    "✅ Built enterprise-ready configuration system",
]

for accomplishment in accomplishments:
    print(f"  {accomplishment}")

print("\n🔮 Next Steps & Extensions:")
next_steps = [
    "📊 Add Chapter 6 sector committee integration",
    "🔄 Implement 4-vintage staggered ensemble",
    "📈 Add performance attribution by sleeve",
    "🌐 Connect to real market data feeds",
    "⚡ Add regime-aware signal adjustments",
    "🏛️ Implement enterprise governance features",
]

for step in next_steps:
    print(f"  {step}")

print("\n💡 The production infrastructure is ready for real-world deployment!")
print("   Replace toy data with live feeds and configure for your use case.")

## 💪 Exercises

### Exercise A: Configuration Impact
Modify the `portfolio_config` parameters and observe how they affect the final portfolio:
- Change `max_sector_weight` from 25% to 15%
- Adjust `target_beta` from 0.0 to 0.5
- Modify `max_cluster_weight` and see the impact on tech exposure

### Exercise B: Signal Sensitivity
Test how the calibration system responds to different signal strengths:
- Create scores with all 1s vs all 5s
- Vary confidence levels and observe μ changes
- Test the impact of `max_z_score` capping

### Exercise C: Risk Analysis
Analyze the portfolio's risk characteristics:
- Calculate the correlation between long and short positions
- Estimate the portfolio's beta to different factors
- Assess concentration risk across different dimensions

### Exercise D: Chapter 6 Integration
Connect this notebook to Chapter 6 outputs:
- Import sector scores from Chapter 6 analysis
- Build a portfolio using real LLM-generated signals
- Compare toy vs real signal portfolio characteristics